In [1]:
!pip install datasets transformers --quiet

In [2]:
import sys
import os
from torch.optim import AdamW
import torch
from tqdm.auto import tqdm
tqdm.pandas()

DRIVE_PATH = "/content/drive/MyDrive/transformers_layers/"
sys.path.append(DRIVE_PATH)

In [3]:
%pdb on

Automatic pdb calling has been turned ON


In [4]:
!pip install -Uqq ipdb
import ipdb

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [6]:
from data.dataloader_superglue import DataSuperglue
import lm_utils
from lm_utils import LM
import constants

In [7]:
class run_model():

  def __init__(self, _db_name, _db_class, _model):
    self.lm: LM = LM(_is_pretrain=False)
    self.train_dataloader, self.val_dataloader, self.test_dataloader = DataSuperglue(self.lm, _db_name, _db_class, device).get_db_dataloaders()
    self.model = _model(self.lm)
    self.optimizer = AdamW(self.model.parameters(), lr=5e-5)
    self.N_EPOCH = 5


  def train(self):
    # ipdb.set_trace()
    self.model.train()
    for epoch in range(self.N_EPOCH):
      for step, batch in tqdm(enumerate(self.train_dataloader)):
          # ipdb.set_trace()
          self.optimizer.zero_grad()
          outputs = self.model(batch)
          loss = outputs.loss
          loss.backward()
          self.optimizer.step()
  
  def eval(self):
    self.model.eval()
    for step, batch in tqdm(enumerate(self.val_dataloader)):
        with torch.no_grad():
          positive_tokenized, negative_tokenized, positive_input, negative_input = batch
          self.model(positive_tokenized, negative_tokenized, positive_input, negative_input)

  def test(self):
    self.model.eval()
    for step, batch in tqdm(enumerate(self.test_dataloader)):
        with torch.no_grad():
          positive_tokenized, negative_tokenized, positive_input, negative_input = batch
          self.model(positive_tokenized, negative_tokenized, positive_input, negative_input)

In [8]:
def init_task(_task):
  db_name = _task
  if _task == constants.CB:
      from data.datasets.cb import CbDataset
      from models.cb_model import CbModel
      db_class= CbDataset
      model = CbModel
  elif _task == constants.COPA:
      from data.datasets.copa import CopaDataset
      from models.copa_model import CopaModel
      db_class= CopaDataset
      model = CopaModel
  else:
      from data.datasets.rte import Rteataset
      from models.rte_model import RteModel
      db_name = constants.RTE
      db_class= RteDataset
      model = RteModel
  return db_name, db_class, model

In [9]:
db_name, db_class, model = init_task("cb")
train_model = run_model(db_name, db_class, model)
train_model.train()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Reusing dataset super_glue (/root/.cache/huggingface/datasets/super_glue/cb/1.0.2/d040c658e2ddef6934fdd97deb45c777b6ff50c524781ea434e7219b56a428a7)


  0%|          | 0/3 [00:00<?, ?it/s]

0it [00:00, ?it/s]


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 332, in set_trace
    sys.settrace(self.trace_dispatch)



> /content/drive/MyDrive/transformers_layers/models/cb_model.py(21)forward()
     20 
---> 21         _inputs, _labels = _batch
     22         cls_inputs = self.model(_inputs.input_ids, attention_mask=_inputs.attention_mask).last_hidden_state[:,0,:]

ipdb> _inputs, _labels = _batch
ipdb> _inputs
{'input_ids': tensor([[    0,   250,    35,  ...,     1,     1,     1],
        [    0,   387,    35,  ...,   103,   464,     2],
        [    0,   387,    35,  ...,     1,     1,     1],
        ...,
        [    0,   387,    35,  ...,     1,     1,     1],
        [    0,  1711,  1559,  ...,     1,     1,     1],
        [    0, 49519,  4528,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
ipdb> cls_inputs = self.model(_inputs.input_ids, attention_mask=_inputs.attention_mask).


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 357, in set_quit
    sys.settrace(None)



In [ ]:
!zip 